In [4]:
!pip install faiss-cpu

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [4]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [5]:
"""
VectorStoreRetrieverMemory将记忆存储在VectorDB中，并在每次调用时查询最重要的K个文档。

与大多数其他记忆类不同的是，它不明确跟踪交互的顺序。

在这种情况下，“文档”是先前的对话片段。这可以用来提到AI在对话中早期被告知的相关信息。
"""
from datetime import datetime
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
 

In [7]:
"""
初始化您的VectorStore
"""
import faiss
 
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
 
embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
# embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(OpenAIEmbeddings(), index, InMemoryDocstore({}), {})
 

In [9]:
"""
创建您的VectorStoreRetrieverMemory
记忆体对象是从任何VectorStoreRetriever实例化的。
"""
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)
 
# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) # 
 

In [10]:
# Notice the first result returned is the memory pertaining to tax help, which the language model deems more semantically relevant
# to a 1099 than the other documents, despite them both containing numbers.
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])
 

input: My favorite sport is soccer
output: ...


In [ ]:
""" output
input: My favorite sport is soccer
output: ...
"""

In [11]:
"""
在链中使用
"""
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
{history}
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


"Hello Perry! I'm doing well, thank you. How about you?"

In [12]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="what's my favorite sport?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


'Your favorite sport is soccer.'

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: what's my favorite sport?
AI:[0m

[1m> Finished chain.[0m

'Your favorite sport is soccer.'
"""

In [13]:
# Even though the language model is stateless, since relavent memory is fetched, it can "reason" about the time.
# Timestamping memories and data is useful in general to let the agent determine temporal relevance
conversation_with_summary.predict(input="Whats my favorite food")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: Whats my favorite food
AI:

> Finished chain.


'Your favorite food is pizza.'

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: My favorite food is pizza
output: that's good to know
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: Whats my favorite food
AI:[0m

[1m> Finished chain.[0m

'Your favorite food is pizza.'
"""

In [14]:
# The memories from the conversation are automatically stored,
# since this query best matches the introduction chat above,
# the agent is able to 'remember' the user's name.
conversation_with_summary.predict(input="What's my name?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: Hi, my name is Perry, what's up?
response: Hello Perry! I'm doing well, thank you. How about you?
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: What's my name?
AI:

> Finished chain.


'Your name is Perry.'

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
 
Relevant pieces of previous conversation:
input: Hi, my name is Perry, what's up?
response: Hello Perry! I'm doing well, thank you. How about you?
 
(You do not need to use these pieces of information if not relevant)
 
Current conversation:
Human: What's my name?
AI:[0m

[1m> Finished chain.[0m

'Your name is Perry.'
"""